In [11]:
!pip install datasets

In [12]:
!jupyter labextension enable @jupyter-widgets/jupyterlab-widgets
!conda install -c conda-forge ipywidgets  # For conda environments
!pip install --upgrade ipywidgets


CondaValueError: invalid package specification: #



  Using cached ipywidgets-8.1.2-py3-none-any.whl.metadata (2.4 kB)
  Using cached widgetsnbextension-4.0.10-py3-none-any.whl.metadata (1.6 kB)
  Using cached jupyterlab_widgets-3.0.10-py3-none-any.whl.metadata (4.1 kB)
Using cached ipywidgets-8.1.2-py3-none-any.whl (139 kB)
Using cached jupyterlab_widgets-3.0.10-py3-none-any.whl (215 kB)
Using cached widgetsnbextension-4.0.10-py3-none-any.whl (2.3 MB)
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.6
    Uninstalling widgetsnbextension-3.6.6:
      Successfully uninstalled widgetsnbextension-3.6.6
  Attempting uninstall: jupyterlab-widgets
    Found existing installation: jupyterlab-widgets 1.1.7
    Uninstalling jupyterlab-widgets-1.1.7:
      Successfully uninstalled jupyterlab-widgets-1.1.7
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.5
    Uninstalling ipywidgets-7.7.5:
      Successfully uninstalled ipywidgets-7.7.5


In [7]:
from datasets import get_dataset_config_names

xtreme_subsets = get_dataset_config_names("xtreme")
print(f"XTREME has {len(xtreme_subsets)}")

XTREME has 183


### XTREME (Cross-Lingual Transfer Evaluation of Multilingual Encoders): 
##### It is a widely accepted benchmark in the domain of Cross-Lingual TRansfer Evaluation of Multilingual Encoders (XTREME). XTREME is a comprehensive benchmark dataset for evaluating multilingual named entity recognition (NER) models. It encompasses various languages, allowing researchers to assess how well NER models generalize across them.

### PAN-X dataset: 
##### Consists of Wikipedia articles in many languages, including the 4 most commonly spoken languages in Switzerland, German (62.9%), French (22.9%), Italian (8.4%) and English (5.9%). 

In [8]:
panx_subsets = [s for s in xtreme_subsets if s.startswith("PAN")]
panx_subsets[:3]

['PAN-X.af', 'PAN-X.ar', 'PAN-X.bg']

In [13]:
from datasets import load_dataset

load_dataset("xtreme" , name = "PAN-X.de")

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags', 'langs'],
        num_rows: 20000
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'langs'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'langs'],
        num_rows: 10000
    })
})

In [19]:
from collections import defaultdict
from datasets import DatasetDict

langs = ["de" , "fr" , "it" , "en"]
fracs = [0.629 , 0.229 , 0.084 , 0.059]

# Return a DatasetDict if a key doesn't exist

panx_ch = defaultdict(DatasetDict)

for lang, frac in zip(langs, fracs):
    # Load monolingual corpus
    ds = load_dataset("xtreme" , name = f"PAN-X.{lang}")
    # SHUFFLE AND DOWNSAMPLE EACH SPLIT ACCORDING TO SPOKEN PROPORTION
    for split in ds:
        panx_ch[lang][split] = (
            ds[split].shuffle(seed = 0).select(range(int(frac * ds[split].num_rows)))
        )

Generating train split:   0%|          | 0/20000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/20000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/20000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [20]:
import pandas as pd

pd.DataFrame({lang: [panx_ch[lang]["train"].num_rows] for lang in langs},
            index = ["Number of training examples"])

,de,fr,it,en
Number of training examples,12580,4580,1680,1180


In [21]:
element = panx_ch["de"]["train"][0]
for key, value in element.items():
    print(f"{key} : {value}")

tokens : ['2.000', 'Einwohnern', 'an', 'der', 'Danziger', 'Bucht', 'in', 'der', 'polnischen', 'Woiwodschaft', 'Pommern', '.']
ner_tags : [0, 0, 0, 0, 5, 6, 0, 0, 5, 5, 6, 0]
langs : ['de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de']


In [22]:
for key, value in panx_ch["de"]["train"].features.items():
    print(f"{key}: {value}")

tokens: Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)
ner_tags: Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None), length=-1, id=None)
langs: Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)


In [23]:
tags = panx_ch["de"]["train"].features["ner_tags"].feature
print(tags)

ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None)


In [27]:
def create_tag_names(batch):
    return {"ner_tags_str" : [tags.int2str(idx) for idx in batch["ner_tags"]]}

panx_de = panx_ch["de"].map(create_tag_names)

In [28]:
de_example = panx_de["train"][0]
pd.DataFrame(
    [de_example["tokens"] , de_example["ner_tags_str"]] , 
    ['Tokens' , 'Tags']
)
    

,0,1,2,3,4,5,6,7,8,9,10,11
Tokens,2.000,Einwohnern,an,der,Danziger,Bucht,in,der,polnischen,Woiwodschaft,Pommern,.
Tags,O,O,O,O,B-LOC,I-LOC,O,O,B-LOC,B-LOC,I-LOC,O


In [30]:
from collections import Counter

split2freqs = defaultdict(Counter)
for split, dataset in panx_de.items():
    for row in dataset["ner_tags_str"]:
        for tag in row:
            if tag.startswith("B"):
                tag_type = tag.split("-")[1]
                split2freqs[split][tag_type] += 1
pd.DataFrame.from_dict(split2freqs , orient = "index")

,LOC,ORG,PER
train,6186,5366,5810
validation,3172,2683,2893
test,3180,2573,3071


#### LOC: Location
#### ORG: Organization
#### PER: Person

## Tokenization

In [33]:
!pip install transformers
!pip install accelerate
!pip install train

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for train: filename=train-0.0.5-py3-none-any.whl size=8514 sha256=732d0ebecaba3ba26d7f8e9bc347f92adb109bc2576a388edf22749985f0f771
  Stored in directory: c:\users\maste\appdata\local\pip\cache\wheels\2d\8b\d8\d8fcaee413eeb0fb4d5dc40dcd8a60582da5c7d5e66874f10d
Successfully built train


In [34]:
from transformers import AutoTokenizer

bert_model_name = "bert-base-cased"
xlmr_model_name = "xlm-roberta-base"
bert_tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
xlmr_tokenizer = AutoTokenizer.from_pretrained(xlmr_model_name)

In [35]:
text = "Kartikey Bartwal loves cute girls!"
bert_tokens = bert_tokenizer(text).tokens()
xlmr_tokens = xlmr_tokenizer(text).tokens()

In [37]:
bert_tokens

['[CLS]',
 'Ka',
 '##rt',
 '##ike',
 '##y',
 'Bart',
 '##wal',
 'loves',
 'cute',
 'girls',
 '!',
 '[SEP]']

In [36]:
xlmr_tokens

['<s>',
 '▁Kar',
 'ti',
 'key',
 '▁Bart',
 'wal',
 '▁love',
 's',
 '▁cute',
 '▁girls',
 '!',
 '</s>']

## Creating a Custom Model for Token Classification

#### Here token classification means determining the named entity of a given token in the language text

In [39]:
import torch.nn as nn
from transformers import XLMRobertaConfig
from transformers.modeling_outputs import TokenClassifierOutput
from transformers.models.roberta.modeling_roberta import RobertaModel
from transformers.models.roberta.modeling_roberta import RobertaPreTrainedModel

In [63]:
class XLMRobertaForTokenClassification(RobertaPreTrainedModel):
    config_class = XLMRobertaConfig

    def __init__(self , config):
        super().__init__(config)
        self.num_labels = config.num_labels

        # LOAD MODEL       'BODY'
        self.roberta = RobertaModel(config , add_pooling_layer = False)

        # SET UP THE TOKEN CLASSIFICATION       'HEAD'
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size , config.num_labels)

        # LOAD AND INITIALIZE WEIGHTS
        self.init_weights()

    def forward(self , input_ids = None , attention_mask = None, token_type_ids = None, labels = None ,  **kwargs):
        # USE MODEL BODY TO GET ENCODER REPRESENTATION
        outputs = self.roberta(input_ids , attention_mask = attention_mask , token_type_ids = token_type_ids , **kwargs)

        # APPLY CLASSIFIER TO ENCODER REPRESENTATION
        sequence_output = self.dropout(outputs[0])
        logits = self.classifier(sequence_output)

        # CALCULATE LOSSES
        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1 , self.num_labels) , labels.view(-1))

        # RETURN MODEL OUTPUT OBJECT
        return TokenClassifierOutput(
            loss = loss,
            logits = logits,
            hidden_states = outputs.hidden_states,
            attentions = outputs.attentions
        )

### Loading a Custom Model

In [64]:
index2tag = {idx: tag for idx, tag in enumerate(tags.names)}
tags2index = {tag: idx for idx, tag in enumerate(tags.names)}

In [65]:
from transformers import AutoConfig

xlmr_config = AutoConfig.from_pretrained(
    xlmr_model_name,
    num_labels = tags.num_classes,
    id2label = index2tag,
    label2id = tags2index
)

In [66]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
xlmr_model = (
    XLMRobertaForTokenClassification.from_pretrained(xlmr_model_name , config = xlmr_config).to(device)
)

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [67]:
input_ids = xlmr_tokenizer.encode(text , return_tensors = "pt")
pd.DataFrame([xlmr_tokens , input_ids[0].numpy()] , index = ["Tokens" , "Input IDs"])

,0,1,2,3,4,5,6,7,8,9,10,11
Tokens,<s>,▁Kar,ti,key,▁Bart,wal,▁love,s,▁cute,▁girls,!,</s>
Input IDs,0,3423,118,19770,41272,8202,5161,7,99942,29526,38,2


In [68]:
outputs = xlmr_model(input_ids.to(device)).logits
predictions = torch.argmax(outputs , dim = -1)
print(f"Number of tokens in sequece: {len(xlmr_tokens)}")
print(f"Shape of outputs: {outputs.shape}")

Number of tokens in sequece: 12
Shape of outputs: torch.Size([1, 12, 7])


In [69]:
preds = [tags.names[p] for p in predictions[0].cpu().numpy()]
pd.DataFrame([xlmr_tokens , preds] , index = ["Tokens" , "Tags"])

,0,1,2,3,4,5,6,7,8,9,10,11
Tokens,<s>,▁Kar,ti,key,▁Bart,wal,▁love,s,▁cute,▁girls,!,</s>
Tags,O,B-PER,O,O,O,O,O,O,O,O,O,O


### Tokenizing Text for Named Entity Recognition


In [74]:
words, labels = de_example["tokens"], de_example["ner_tags"]

tokenized_input = xlmr_tokenizer(de_example["tokens"] , is_split_into_words = True)
tokens = xlmr_tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
pd.DataFrame([tokens] , index = ["Tokens"])

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
Tokens,<s>,▁2.000,▁Einwohner,n,▁an,▁der,▁Dan,zi,ger,▁Buch,...,▁Wo,i,wod,schaft,▁Po,mmer,n,▁,.,</s>


In [75]:
word_ids = tokenized_input.word_ids()
pd.DataFrame([tokens , word_ids] , index = ["Tokens" , "Word IDs"])

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
Tokens,<s>,▁2.000,▁Einwohner,n,▁an,▁der,▁Dan,zi,ger,▁Buch,...,▁Wo,i,wod,schaft,▁Po,mmer,n,▁,.,</s>
Word IDs,None,0,1,1,2,3,4,4,4,5,...,9,9,9,9,10,10,10,11,11,None


In [77]:
previous_word_idx = None
label_ids = []

for word_idx in word_ids:
    if word_idx is None or word_idx == previous_word_idx:
        label_ids.append(-100)
    elif word_idx != previous_word_idx:
        label_ids.append(labels[word_idx])
    previous_word_idx = word_idx

In [78]:
labels = [index2tag[l] if l != -100 else "IGN" for l in label_ids]
index = ["Tokens" , "Word IDs" , "Label IDs" , "Labels"]

In [79]:
pd.DataFrame([tokens , word_ids , label_ids , labels] , index = index)

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
Tokens,<s>,▁2.000,▁Einwohner,n,▁an,▁der,▁Dan,zi,ger,▁Buch,...,▁Wo,i,wod,schaft,▁Po,mmer,n,▁,.,</s>
Word IDs,None,0,1,1,2,3,4,4,4,5,...,9,9,9,9,10,10,10,11,11,None
Label IDs,-100,0,0,-100,0,0,5,-100,-100,6,...,5,-100,-100,-100,6,-100,-100,0,-100,-100
Labels,IGN,O,O,IGN,O,O,B-LOC,IGN,IGN,I-LOC,...,B-LOC,IGN,IGN,IGN,I-LOC,IGN,IGN,O,IGN,IGN


In [89]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = xlmr_tokenizer(examples["tokens"] , truncation =  True , is_split_into_words = True)

    labels = []

    for idx, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index = idx)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None or word_idx == previous_word_idx:
                label_ids.append(-100)
            else:
                label_ids.append(label[word_idx])
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels

    return tokenized_inputs

In [90]:
def encode_panx_dataset(corpus):
    return corpus.map(
        tokenize_and_align_labels , batched = True, remove_columns = ["langs" , "ner_tags" , "tokens"]
    )

In [91]:
panx_de_encoded = encode_panx_dataset(panx_ch["de"])

Map:   0%|          | 0/12580 [00:00<?, ? examples/s]

Map:   0%|          | 0/6290 [00:00<?, ? examples/s]

Map:   0%|          | 0/6290 [00:00<?, ? examples/s]

## Performance Measures

In [93]:
!pip install seqeval

     ---------------------------------------- 0.0/43.6 kB ? eta -:--:--
     --------- ------------------------------ 10.2/43.6 kB ? eta -:--:--
     -------------------------- ----------- 30.7/43.6 kB 259.2 kB/s eta 0:00:01
     -------------------------------------- 43.6/43.6 kB 303.3 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16185 sha256=82281bc060f376e9eb20366ab589e58f7bc3fbedafedeebda8fe7746febd33af
  Stored in directory: c:\users\maste\appdata\local\pip\cache\wheels\bc\92\f0\243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval


In [99]:
from seqeval.metrics import classification_report
import numpy as np

def align_predictions(predictions , label_ids):
    preds = np.argmax(predictions , axis = 2)
    batch_size , seq_len = preds.shape
    labels_list , preds_list = [] , []

    for batch_idx in range(batch_size):
        example_labels , example_preds = [] , []
        for seq_idx in range(seq_len):
            # IGNORE LABEL IDS = -100
            if label_ids[batch_idx , seq_idx] != -100:
                example_labels.append(index2tag[label_ids[batch_idx][seq_idx]])
                example_preds.append(index2tag[preds[batch_idx][seq_idx]])
        labels_list.append(example_labels)
        preds_list.append(example_preds)

    return preds_list , labels_list

In [100]:
# Sample predictions and label_ids
predictions = np.array([[[0.1, 0.2, 0.7], [0.3, 0.3, 0.4], [0.5, 0.3, 0.2]],
                         [[0.7, 0.2, 0.1], [0.2, 0.3, 0.5], [0.4, 0.3, 0.3]]])

label_ids = np.array([[0, 1, -100], [2, 1, 0]])

# Example index to tag mapping
index2tag = {0: 'O', 1: 'B-PER', 2: 'I-PER'}

# Call the align_predictions function
preds_list, labels_list = align_predictions(predictions, label_ids)

# Print the results
print("Predictions:")
for preds in preds_list:
    print(preds)

print("\nLabels:")
for labels in labels_list:
    print(labels)

print("\Classification Report:")
for labels in labels_list:
    print(labels)

Predictions:
['I-PER', 'I-PER']
['O', 'I-PER', 'O']

Labels:
['O', 'B-PER']
['I-PER', 'B-PER', 'O']


In [103]:
# Example ground truth labels and predictions
ground_truth = [['B-PER', 'I-PER', 'O', 'O', 'B-LOC', 'I-LOC'],
                ['O', 'B-ORG', 'I-ORG', 'O']]
predictions = [['I-PER', 'I-PER', 'O', 'O', 'I-LOC', 'I-LOC'],
               ['O', 'B-ORG', 'B-ORG', 'O']]

# Calculate the classification report
report = classification_report(ground_truth, predictions)

# Print the report
print(report)

              precision    recall  f1-score   support

         LOC       1.00      1.00      1.00         1
         ORG       0.00      0.00      0.00         1
         PER       1.00      1.00      1.00         1

   micro avg       0.50      0.67      0.57         3
   macro avg       0.67      0.67      0.67         3
weighted avg       0.67      0.67      0.67         3



## Fine-Tuning XLM-RoBERTa

In [106]:
from transformers import TrainingArguments

num_epochs = 3
batch_size = 24
logging_steps = len(panx_de_encoded["train"]) // batch_size
model_name = f"{xlmr_model_name}-finetuned-panx-de"
training_args = TrainingArguments(
    output_dir = model_name , log_level = "error" , num_train_epochs = num_epochs,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size , evaluation_strategy = "epoch",
    save_steps = 1e6,
    weight_decay = 0.01,
    disable_tqdm = False,
    logging_steps = logging_steps,
    push_to_hub = True
)

In [107]:
from huggingface_hub import notebook_login

notebook_login()

In [113]:
from seqeval.metrics import f1_score

def compute_metrics(eval_pred):
    y_pred, y_true = align.predictions(eval_pred.predictions, eval_pred.label_ids)
    return {"f1" : f1_score(y_true , y_pred)}

In [114]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(xlmr_tokenizer)

In [115]:
def model_init():
    return (
        XLMRobertaForTokenClassification.from_pretrained(xlmr_model_name , config = xlmr_config).to(device)
    )

In [116]:
from transformers import Trainer

trainer = Trainer(
    model_init = model_init,
    args = training_args,
    data_collator = data_collator,
    compute_metrics = compute_metrics,
    train_dataset = panx_de_encoded["train"],
    eval_dataset = panx_de_encoded["validation"],
    tokenizer = xlmr_tokenizer
)

In [117]:
trainer.train().push_to_hub(commit_message="Training is Completed!")

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 2.00 GiB of which 0 bytes is free. Of the allocated memory 5.19 GiB is allocated by PyTorch, and 123.75 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [118]:
text_de = "Jeff Dean ist ein Informatiker bei Google in Kalifornien"

tag_text(text_de , tags , trainer.model , xlmr_tokenizer)

NameError: name 'tag_text' is not defined

## Error Analysis

### Data Collator: 
##### A data collator is a component responsible for processing and combining individual samples or examples into batches for training or evaluation. It helps in preparing the data for efficient processing by the model.

##### The main tasks of a data collator typically include:

#### 1) Padding: 
###### Ensuring that all samples in a batch have the same length by padding shorter sequences with a special token (e.g., <pad>).
#### 2) Batching: 
###### Grouping individual samples into batches based on a specified batch size.
#### 3) Conversion: 
###### Converting raw inputs (e.g., text or numerical data) into the format expected by the model (e.g., token IDs).
#### 4) Handling Special Tokens:
###### Managing special tokens (e.g., <bos>, <eos>, <pad>) that are used in the model's input.

In [121]:
from torch.nn.functional import cross_entropy

def forward_pass_with_label(batch):
    # CONVERT DICT OF LISTS TO LISTS SUITABLE FOR DATA COLLATOR
    features = [dict(zip(batch , t)) for t in zip(*batch.values())]
    # PAD INPUTS AND LABELS AND PUT ALL TENSORS ON DEVICE
    batch = data_collator(features)
    input_ids = batch["input_ids"].to(device)
    attention_mask = batch["input_ids"].to(device)
    labels = batch["labels"].to(device)
    with torch.no_grad():
        # PASS DATA THROUGH MODEL
        output = trainer.model(input_ids , attention_mask)
        # logit.size: [batch_size , sequence_length , classes]
        # PREDICT CLASS WITH LARGEST LOGIT VALUE ON CLASSES AXIS
        predicted_label = torch.argmax(output.logits , axis = -1).cpu().numpy()
    # CALCULATE LOSS PER TOKEN AFTER FLATTENING BATCH DIMENSION WITH VIEW
    loss = cross_entropy(output.logits.view(-1 , 7), labels.view(-1) , reduction = "none")

    # UNFLATTEN BATCH DIMENSIONS AND CONVERT TO NUMPY ARRAY
    loss = loss.view(len(input_ids), -1).cput().numpy()

    return {"loss":loss , "predicted_label": predicted_label}

In [122]:
valid_set = panx_de_encoded["validation"]
valid_set = valid_set.map(forward_pass_with_label , batched = True, batch_size = 32)
df = valid_set.to_pandas()

Parameter 'function'=<function forward_pass_with_label at 0x0000029A345AF420> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/6290 [00:00<?, ? examples/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 2.00 GiB of which 0 bytes is free. Of the allocated memory 5.19 GiB is allocated by PyTorch, and 123.70 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [124]:
index2tag[-100] = "IGN"
df["input_tokens"] = df["input_ids"].apply(
    lambda x: xlmr_tokenizer.convert_ids_to_tokens(x)
)
df["predicted_label"] = df["predicted_label"].apply(
    lambda x: [index2tag[i] for i in x]
)
df["labels"] = df["labels"].apply(
    lambda x: [index2tag[i] for i in x]
)
df["loss"] = df.apply(
    lambda x: x["loss"][:len(x['input_ids'])], axis = 1
)
df["predicted_label"] = df.apply(
    lambda x: x["predicted_label"][:len(x['input_ids'])], axis = 1
 )

NameError: name 'df' is not defined

In [ ]:
df_tokens = df.apply(pd.Series.explode)
df_tokens = df_tokens.query("labels != 'IGN'")
df_tokens["loss"] = df_tokens["loss"].astype(float).round(2)
df_tokens.head(7)

In [125]:
df_tokens.groupby("input_tokens")[["loss"]].agg(["count" , "mean" , "sum"]).droplevel(level = 0, axis = 1)
# GET RID OF multi-level COLUMNS

NameError: name 'df_tokens' is not defined